# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

Таблица содержит 12 столбцов и 21525 строк c типом данных в скобках:
    
* children — количество детей в семье (int64)
* days_employed — общий трудовой стаж в днях (float64)
* dob_years — возраст клиента в годах (int64)
* education — уровень образования клиента (object)
* education_id — идентификатор уровня образования (int64)
* family_status — семейное положение (object)
* family_status_id — идентификатор семейного положения (int64)
* gender — пол клиента (object)
* income_type — тип занятости (object)
* debt — имел ли задолженность по возврату кредитов (int64)
* total_income — ежемесячный доход (float64)
* purpose — цель получения кредита (object)

Количество значения в столбцах days_employed и total_income говорит о том, что есть пропуски

In [143]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

1. В столбцах days_employed и total_income были обнаружены пропущенные значения типа float64. 
2. Доля пропущенных значений в каждом из столбцов с пропусками составляет около 10%. 
3. Возможными причинами появления пропуска в столбце days_employed является отсутствие трудового стажа, что могло бы означать, что человек либо не работает, либо работает не официально, поэтому пропуски и в столбце total_income.
4. Пропущенные значения лучше заполнить медианным значением, т.к в выборке присутствуют абсолютно разные числовые значения.

In [144]:
days_employed_median = df['days_employed'].median() # Находим медианное значение
total_income_median = df['total_income'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median) # Заменяем пропуски
df['total_income'] = df['total_income'].fillna(total_income_median)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [145]:
display(df['children'].unique()) # Есть минусовые значения, возможно неправильно занесли данные
df['children'] = df['children'].replace(-1,1) # Заменяем значение в столбце children -1 на 1 
df['children'] = df['children'].replace(20,2) # Заменяем значение в столбце children 20 на 2
display(df['children'].unique()) # Проверка результата

array([ 1,  0,  3,  2, -1,  4, 20,  5])

array([1, 0, 3, 2, 4, 5])

In [146]:
display(df['days_employed'].unique()) # Все минусовые значения, при заполнении произошла  техническая ошибка
df['days_employed'] = df['days_employed'].abs() # Заменяем отрицательные значения методом abs
display(df['days_employed'].unique()) # Проверка результата

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

array([8437.67302776, 4024.80375385, 5623.42261023, ..., 2113.3468877 ,
       3112.4817052 , 1984.50758853])

In [147]:
display(df['dob_years'].unique()) # Возраст должен быть от 18 лет, ошибочно поставили 0
df = df.drop(df[df['dob_years'] == 0].index) # Удаляем все строки, где dob_years - 0
display(df['dob_years'].unique()) # Проверка результата

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

In [148]:
display(df['education'].unique()) # Нужно привести к одному регистру и есть дубликаты, возможной причиной стал человеческий фактор 
# Удаление дубликатов происходят в "0.5  Шаг 2.4. Удаление дубликатов."

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [149]:
display(df['education_id'].unique()) # Все ОК

array([0, 1, 2, 3, 4])

In [150]:
display(df['family_status'].unique()) # Все ОК

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [151]:
display(df['family_status_id'].unique()) # Все ОК

array([0, 1, 2, 3, 4])

In [152]:
display(df['gender'].unique()) # XNA, ошибочно написали
df = df.drop(df[df['gender'] == 'XNA'].index) # Удаляем в столбце gender значение XNA
display(df['gender'].unique()) # Проверка результата

array(['F', 'M', 'XNA'], dtype=object)

array(['F', 'M'], dtype=object)

In [153]:
display(df['income_type'].unique()) # Все ОК

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [154]:
display(df['debt'].unique()) # Все ОК

array([0, 1])

In [155]:
display(df['total_income'].unique()) # Все ОК

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

In [156]:
display(df['purpose'].unique()) # Неявные дубликаты. Категоризированно по ключевым словам в п. "0.8  Шаг 2.7. Категоризация целей кредита."

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

### Шаг 2.3. Изменение типов данных.

In [157]:
try:
    df['total_income']= df['total_income'].astype('int') # Замена вещественного типа на целоцисленный
    df['days_employed'] = df['days_employed'].astype('int')
except Exception as e:
    print('Не удалось преобразовать поле total_income в тип int')

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21423 non-null  int64 
 1   days_employed     21423 non-null  int64 
 2   dob_years         21423 non-null  int64 
 3   education         21423 non-null  object
 4   education_id      21423 non-null  int64 
 5   family_status     21423 non-null  object
 6   family_status_id  21423 non-null  int64 
 7   gender            21423 non-null  object
 8   income_type       21423 non-null  object
 9   debt              21423 non-null  int64 
 10  total_income      21423 non-null  int64 
 11  purpose           21423 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Шаг 2.4. Удаление дубликатов.

In [159]:
df['education'].value_counts() # Подсчет количества До

среднее                13691
высшее                  4686
СРЕДНЕЕ                  770
Среднее                  708
неоконченное высшее      665
ВЫСШЕЕ                   273
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [160]:
df['education'] = df['education'].str.lower() # Приводим к нижнему регистру

In [161]:
df['education'].value_counts() # Подсчет количества После

среднее                15169
высшее                  5225
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

In [162]:
df.duplicated().sum() # Поиск дубликатов во всем датафрейме до устранения

71

In [163]:
df = df.drop_duplicates().reset_index(drop = True)

In [164]:
df.duplicated().sum() # Поиск дубликатов во всем датафрейме после устранения

0

1. Дубликаты в поле education появились из-за несоблюдения регистров при заполнении данных. Для устранения дубликатов все строки столбца education были приведены к нижнему регистру.
2. Во всем датафрейме было 71 дублей, они тоже были устранены методом drop_duplicates.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [165]:
print(df['education'].unique()) # Вывод уникальных значений
print(df['education_id'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
[0 1 2 3 4]


In [166]:
print(df['family_status'].unique())
print(df['family_status_id'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
[0 1 2 3 4]


In [167]:
try:
    del df['education']
    del df['family_status']
except Exception as e:
    print(e.__str__())

In [168]:
data_education = [['высшее', 'среднее', 'неоконченное высшее', 'начальное', 'ученая степень']]
columns_education = [0, 1, 2, 3, 4]
df_education = pd.DataFrame(data = data_education, columns = columns_education)
display(df_education)

,0,1,2,3,4
0,высшее,среднее,неоконченное высшее,начальное,ученая степень


In [169]:
data_family_status = [['женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'Не женат / не замужем']]
columns_family_status = [0, 1, 2, 3, 4]
df_family_status = pd.DataFrame(data = data_family_status, columns = columns_family_status)
display(df_family_status)

,0,1,2,3,4
0,женат / замужем,гражданский брак,вдовец / вдова,в разводе,Не женат / не замужем


### Шаг 2.6. Категоризация дохода.

In [170]:
def get_total_income_category(total_income): # Создаем функцию
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'

In [171]:
get_total_income_category(25000) # Проверяем работу функции при разных значениях на входе

'E'

In [172]:
get_total_income_category(235000) # Проверяем работу функции при разных значениях на входе

'B'

In [173]:
df['total_income_category'] = df['total_income'].apply(get_total_income_category) # Создаем столбец

### Шаг 2.7. Категоризация целей кредита.

In [174]:
display(df['purpose'].unique()) # Неявные дубликаты, причина - человеческий фактор

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [175]:
def get_purpose_category(purpose): # Создаем функцию
    """
    'операции с автомобилем',
    'операции с недвижимостью',
    'проведение свадьбы',
    'получение образования'.
    """
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if ('жиль' in purpose) or ('недвижимост' in purpose):
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образовани' in purpose:
        return 'получение образования'

In [176]:
get_purpose_category('на покупку автомобиля') # Проверяем работу функции при разных значениях на входе

'операции с автомобилем'

In [177]:
df['purpose_category'] = df['purpose'].apply(get_purpose_category) # Проверяем работу функции при разных значениях на входе

In [178]:
df['purpose_category'].value_counts() # Подсчет уникальных значений 

операции с недвижимостью    10763
операции с автомобилем       4284
получение образования        3995
проведение свадьбы           2310
Name: purpose_category, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [179]:
#children_grouped = df.groupby('children').agg({'debt':['count','sum']}) # Методом agg() указываем, какие именно функции применить к столбцу debt 
#children_grouped['quality'] = children_grouped['debt']['sum'] / children_grouped['debt']['count'] * 100 # Посчет доли просрочек по кредиту в зависимости от количества детей
#display(children_grouped)

In [180]:
children_count_1 = pd.pivot_table(df, index='children', values='debt', aggfunc='count')
children_sum_1 = pd.pivot_table(df, index='children', values='debt', aggfunc='sum')
quality_children = children_sum_1 / children_count_1 * 100
display(quality_children)

,debt
children,
0,7.545824
1,9.134119
2,9.555345
3,8.231707
4,9.756098
5,0.000000


##### Вывод 1:

Больше всего задолженностей у семей с детьми от 1-4 детей. У семей с 5 детьми нету задолженностей. Это может быть связано с финансами.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [182]:
family_count = pd.pivot_table(df, index='family_status_id', values='debt', aggfunc='count')
family_sum = pd.pivot_table(df, index='family_status_id', values='debt', aggfunc='sum')
quality_family = family_sum / family_count * 100
display(df_family_status)
display(quality_family)

,0,1,2,3,4
0,женат / замужем,гражданский брак,вдовец / вдова,в разводе,Не женат / не замужем


,debt
family_status_id,
0,7.542718
1,9.348511
2,6.498952
3,7.172996
4,9.770938


##### Вывод 2:

У категорий "не женат / не замужем" и "гражданский брак" больше всего задолжностей, можно судить, что они не хотят нести ответственность ни за финансы, ни в жизни, а в категории  "вдовец / вдова" и "в разводе" меньше всего задолжностей, эти люди остались одни, пережили многое и чувство ответственности у них большое. В категории "женат / замужем" средние показатели, вероятнее всего эти люди ответственны, но есть не большие проблемы с финансами.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [184]:
total_income_count = pd.pivot_table(df, index='total_income_category', values='debt', aggfunc='count')
total_income_sum = pd.pivot_table(df, index='total_income_category', values='debt', aggfunc='sum')
quality_total_income = total_income_sum / total_income_count * 100
display(quality_total_income)

,debt
total_income_category,
A,8.000000
B,7.080176
C,8.486483
D,6.034483
E,9.090909


##### Вывод 3:

1. Клиенты, уровень дохода которых 30 000 – 50 000 руб., они умеют рассчитывать свои возможности и траты
2. Клиенты, уровень дохода которых больше 50 000 руб., задолжностей больше вероятнее всего потому что и траты у них больше. 
3. Клиенты, у которых доход составляет до 30 000 руб., у них больше всех задолжностей, вероятно, это связано с низким уровнем дохода.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [186]:
purpose_count = pd.pivot_table(df, index='purpose_category', values='debt', aggfunc='count')
purpose_sum = pd.pivot_table(df, index='purpose_category', values='debt', aggfunc='sum')
quality_purpose = purpose_sum / purpose_count * 100
display(quality_purpose)

,debt
purpose_category,
операции с автомобилем,9.337068
операции с недвижимостью,7.237759
получение образования,9.261577
проведение свадьбы,7.965368


##### Вывод 4:

1. У категорий "операции с недвижимостью" и "проведение свадьбы" меньше всего задолжностей, к недвижимости люди относятся ответственно, т.к срок платежа большой и не хочется остаться в долгах на крупные суммы, это пугает людей. А после свадьбы гости дарят деньги, которые потом можно положить на кредит, поэтому должников меньше.
2. У категорий "операции с автомобилем" и "получение образования" задолжностей больше, может быть потому что люди берут кредит не совсем обдуманно, например поведясь на рекламу машины. Одновременно учиться и работать тяжело, и найти работу с хорошей заработной платой трудно, появляются задолжности из-за недостачи финансов.

## Общий вывод:

По итогам своей работы, могу сказать, что самыми надежными клиентами будут люди, имеющие 5 детей (т.к. они более опытные и ответственные), вдовец или вдова, имеющие уровень дохода 30 000 – 50 000 руб в месяц, которые берут кредит для операции с недвижимостью.

Не надежными кредиторами будут люди, имеющие 1, 2, 4 детей, одиночки, у которых доход составляет до 30 000 руб и целью является покупка автомобиля.